## Libraries

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rc

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

import os
from sktime.utils.data_io import load_from_arff_to_dataframe
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sktime.transformations.panel.tsfresh import TSFreshRelevantFeatureExtractor
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sktime.transformations.panel.compose import ColumnConcatenator

from sktime.datatypes._panel._convert import from_2d_array_to_nested

# Report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Load data


### Open the ARFF file

This ".arff" format was used to load the dataframe structure in a easier way.

In [ ]:
datosIMU, yimu = load_from_arff_to_dataframe(r"C:\Users\milov\Desktop\Mediciones y Matlab\Mediciones Todos\Todos_imuf.arff")
datosPres, ypres = load_from_arff_to_dataframe(r"C:\Users\milov\Desktop\Mediciones y Matlab\Mediciones Todos\Todos_textf.arff")

In [ ]:
print(datosIMU.shape)
datosIMU.head()

In [ ]:
print(datosPres.shape)
datosPres.head()

In [ ]:
clases = np.unique(yimu)
print(clases)

### Train and test split

In [ ]:
pximu_train, pximu_test, pyimu_train, pyimu_test = train_test_split(datosIMU, yimu, test_size=.2)

In [ ]:
pxpres_train, pxpres_test, pypres_train, pypres_test = train_test_split(datosPres, ypres, test_size=.2)

# Feature extraction using TSFresh

### For IMU data

In [ ]:
timu = TSFreshRelevantFeatureExtractor(default_fc_parameters="efficient",n_jobs=16, show_warnings=False)
ximu = timu.fit_transform(datosIMU, yimu)
ximu.head()

### For textile pressure sensors

In [ ]:
tpres = TSFreshRelevantFeatureExtractor(default_fc_parameters="efficient",n_jobs=16, show_warnings=False)
xpres = tpres.fit_transform(datosPres, ypres)
xpres.head()

## Train and test split

### For IMU data

In [ ]:
ximu_train, ximu_test, yimu_train, yimu_test = train_test_split(ximu, yimu, test_size=.2)

In [ ]:
print(ximu_train.shape)
print(yimu_train.shape)

In [ ]:
print(ximu_test.shape)
print(yimu_test.shape)

### For pressure sensors data

In [ ]:
xpres_train, xpres_test, ypres_train, ypres_test = train_test_split(xpres, ypres, test_size=.2)

In [ ]:
print(xpres_train.shape)
print(ypres_train.shape)

In [ ]:
print(xpres_test.shape)
print(ypres_test.shape)

## Classiffication with IMU data

### Algorithm 1: Random Forest Classiffier

In [ ]:
rfimu = RandomForestClassifier(n_jobs=8)
rfimu.fit(ximu_train, yimu_train)
rfimu.score(ximu_test,yimu_test)

In [ ]:
rfimu_pred = rfimu.predict(ximu_test)
cm = confusion_matrix(yimu_test, rfimu_pred)
print(cm)

target_names = ['RSTLST', 'RSTLSW', 'RSWLST']
print(classification_report(yimu_test, rfimu_pred, target_names=target_names))

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}%".format(cm[i, j]*100),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
plot_confusion_matrix(cm,target_names,title='Confusion matrix')

### Algorithm 2: Time Series Forest 

To use this algorithm, its required to transform the dataframe to a nested dataframe.

In [ ]:
x2imu_train = from_2d_array_to_nested(ximu_train)
x2imu_test = from_2d_array_to_nested(ximu_test)

In [ ]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100, random_state=None, n_jobs=8)),
]
concIMU = Pipeline(steps)
concIMU.fit(x2imu_train, yimu_train)
concIMU.score(x2imu_test, yimu_test)

In [ ]:
concIMU_pred = concIMU.predict(x2imu_test)
cm2 =confusion_matrix(yimu_test, concIMU_pred)
print(cm2)

target_names = ['RSTLST2', 'RSTLSW', 'RSWLST']
print(classification_report(yimu_test, concIMU_pred, target_names=target_names))

In [ ]:
plot_confusion_matrix(cm2,target_names,title='Confusion matrix')

## Algorithm 3: Mr-SEQL

In [ ]:
seqlimu = MrSEQLClassifier(seql_mode='clf')
seqlimu.fit(ximu_train, yimu_train)
seqlimu.score(ximu_test, yimu_test)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

seqlimu_pred = seqlimu.predict(ximu_test)
cm3 =confusion_matrix(yimu_test, seqlimu_pred)
print(cm3)
target_names = ['RSTLST', 'RSTLSW', 'RSWLST']
print(classification_report(yimu_test, seqlimu_pred, target_names=target_names))

In [ ]:
plot_confusion_matrix(cm3,target_names,title='Confusion matrix')

## Classification with pressure sensors data

### Algorithm 1: Random Forest Classiffier

In [ ]:
rfpres = RandomForestClassifier(n_jobs=8)
rfpres.fit(xpres_train, ypres_train)
rfpres.score(xpres_test,ypres_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

rfpres_pred = rfpres.predict(xpres_test)
cm4 =confusion_matrix(ypres_test, rfpres_pred)
print(cm4)
target_names = ['RSTLST', 'RSTLSW', 'RSWLST']
print(classification_report(ypres_test, rfpres_pred, target_names=target_names))

In [ ]:
plot_confusion_matrix(cm4,target_names,title='Confusion matrix')

### Algorithm 2: Time Series Forest

To use this algorithm, its required to transform the dataframe to a nested dataframe.

In [ ]:
x2pres_train = from_2d_array_to_nested(xpres_train)
x2pres_test = from_2d_array_to_nested(xpres_test)

In [ ]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100, random_state=None, n_jobs=8)),
]
concPres = Pipeline(steps)
concPres.fit(x2pres_train, ypres_train)
concPres.score(x2pres_test, ypres_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

concpres_pred = concPres.predict(x2pres_test)
cm5 =confusion_matrix(ypres_test, concpres_pred)
print(cm5)
target_names = ['RSTLST', 'RSTLSW', 'RSWLST']
print(classification_report(ypres_test, concpres_pred, target_names=target_names))

In [ ]:
plot_confusion_matrix(cm5,target_names,title='Confusion matrix')

### Algorithm 3: Mr-SEQL


In [ ]:
seqlpres = MrSEQLClassifier(seql_mode='clf')
seqlpres.fit(xpres_train, ypres_train)
seqlpres.score(xpres_test, ypres_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

seqlpres_pred = seqlpres.predict(xpres_test)
cm6 =confusion_matrix(ypres_test, seqlpres_pred)
print(cm6)
target_names = ['RSTLST', 'RSTLSW', 'RSWLST']
print(classification_report(ypres_test, seqlpres_pred, target_names=target_names))

In [ ]:
plot_confusion_matrix(cm6,target_names,title='Confusion matrix')